## Polarity with Anne Frank's journal entries
This notebook includes functions that takes in journal entries into a nlp model and returns the individual and cummulative polarities for each journal entries and each sentence in a given journal entry (temporarily indexed by date).

In [169]:
# imports
from bs4 import BeautifulSoup
import urllib.request
import re
import json
from textblob import TextBlob
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer

In [170]:
# retrieving Anne Frank data for polarity testing 
def retrieve_data_from_url(url):
    list_8 = []
    dict_8 = {}
    r= urllib.request.urlopen(url).read()
    soup = BeautifulSoup(r, 'lxml')

    for points in soup.find_all('pre'):
        point = str(points.text)
    
    lst = point.split("\n")
    lst = [i for i in lst if i]
    lst = lst[229:]

    r = re.compile("[A-Z]+, [A-Z]+ [0-9]{2}[ ]?,[ ]?[0-9]{4}.*")
    filtered_list = list(filter(r.match, lst))
    filtered_lst_index = []
    
    
    diary_dict = dict.fromkeys(filtered_list)
    for i in filtered_list:
        filtered_lst_index.append(lst.index(i))
        
    dict_index = 0
    
    for i in range(len(filtered_lst_index) - 1):
        beginning = filtered_lst_index[i] + 1
        end = filtered_lst_index[i + 1]
        entry = lst[beginning: end]
        string = ''.join(entry)
        diary_dict[filtered_list[dict_index]] = string
        dict_index = dict_index + 1
        
    filtered = {k: v for k, v in diary_dict.items() if v is not None}
    diary_dict.clear()
    diary_dict.update(filtered)
    diary_sent = diary_dict.copy()
    for i in diary_keys:
        diary_sent[i] = diary_sent[i].split(".")
    return diary_dict, diary_sent

diary_dict, diary_sent = retrieve_data_from_url('https://archive.org/stream/AnneFrankTheDiaryOfAYoungGirl_201606/Anne-Frank-The-Diary-Of-A-Young-Girl_djvu.txt')



In [171]:
# each entry
def entry_polarity(diary_dict):
    diary_keys = list(diary_dict.keys())
    return_dict = dict.fromkeys(diary_keys)
    entry_dict = dict.fromkeys(diary_keys)
    
    sia = SentimentIntensityAnalyzer()
    # polarity for each entry
    for i in diary_keys:
        entry = diary_dict[i]
        temp_dict = sia.polarity_scores(entry)
        edu = TextBlob(entry)
        x = edu.sentiment.polarity
        temp_dict['cummulative'] = x
        return_dict[i] = temp_dict

    return return_dict

entry_polarity = entry_polarity(diary_dict)        

In [181]:
# each sentence in an entry
def sentence_polarity(entry):
    sentences = diary_sent[entry]
    return_dict = {entry: None}
    sentence_dict = dict.fromkeys(sentences)
    return_dict[entry] = sentence_dict

    for i in sentences:
        temp_dict = sia.polarity_scores(i)
        edu = TextBlob(i)
        x = edu.sentiment.polarity
        temp_dict['cummulative'] = x
        return_dict[entry][i] = temp_dict
        
    return return_dict
    
sentence_polarity = sentence_polarity('SATURDAY, JUNE 20,1942 ')  

